In [1]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [1]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random
from multiprocessing import Pool        # Multiprocess Runs
from tqdm import tqdm_notebook, tqdm

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold

sys.path.insert(0, os.path.abspath('../'))
from module.prepare_data import *
# warnings.filterwarnings('ignore')

pd.set_option('max_columns', 200)
pd.set_option('max_rows', 2000)

# tf.config.list_physical_devices('GPU')
tf.test.is_gpu_available()

C:\Users\wangz\.conda\envs\tf_gpu1.14\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\wangz\.conda\envs\tf_gpu1.14\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\wangz\.conda\envs\tf_gpu1.14\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\wangz\.conda\envs\tf_gpu1.14\lib\site-packages\tensorflow\

True

In [2]:
########################### Vars
#################################################################################
VER = 4                          # Our model version
SEED = 42
TARGET      = 'sales'            # Our target
START_TRAIN = 0                  # We can skip some rows (Nans/faster training)
END_TRAIN   = 1913               # End day of our train set
P_HORIZON   = 28                 # Prediction horizon

n_fold = 3
part_len = (END_TRAIN-START_TRAIN)//n_fold

#PATHS for Features
ORIGINAL = '../input/m5-forecasting-accuracy/'
BASE     = '../cache/ori_grid_part_1.pkl'
PRICE    = '../cache/ori_grid_part_2.pkl'
CALENDAR = '../cache/ori_grid_part_3.pkl'
LAGS     = '../cache/ori_lags_df_28.pkl'
MEAN_ENC = '../cache/ori_mean_encoding_df.pkl'
BASE_PATH  = f'../cache/ver{VER}'
BASE_GRID = f'../cache/ver{VER}/base_grid.pkl'

#STORES ids
ORIGINAL = '../input/m5-forecasting-accuracy/'
STORES_IDS = pd.read_csv(ORIGINAL+'sales_train_validation.csv')['store_id']
STORES_IDS = list(STORES_IDS.unique())
TEST_STORE_ID = 'CA_1'
REMOVE_FEATURES = ['d', 'id', 'sales', 'state_id', 'store_id', 'wm_yr_wk']
BASE_FEATURES = ['id','item_id','dept_id','cat_id','store_id', 'state_id','d','sales','release','sell_price','price_max','price_min','price_std','price_mean','event_name_1','event_type_1','event_name_2','event_type_2','snap_CA','snap_TX','snap_WI','tm_d','tm_w','tm_m','tm_y','tm_wm','tm_dw','tm_w_end']
TS_COLUMNS = ['sell_price','sales','event_name_1','event_type_1','event_name_2','event_type_2','snap_CA','snap_TX','snap_WI','tm_d','tm_w','tm_m','tm_y','tm_wm','tm_dw','tm_w_end']
STATIC_COLUMNS = []

In [3]:
try:
    if not os.path.exists(BASE_PATH):
        os.makedirs(BASE_PATH)
except OSError:
    print("Creation of the directory %s failed" % BASE_PATH)
else:
    print("Successfully created the directory %s" % BASE_PATH)

Successfully created the directory ../cache/ver4


In [4]:
########################### Helper to load data by store ID
#################################################################################
# Read data
def get_data_by_store(store):
    
    # Read and contact basic feature
    if not os.path.exists(BASE_GRID):
        grid_df = pd.concat([pd.read_pickle(BASE),
                        pd.read_pickle(PRICE).iloc[:,2:],
                        pd.read_pickle(CALENDAR).iloc[:,2:]],
                        axis=1)
        grid_df['item_id'] = grid_df['item_id'].apply(lambda x: int(x.split('_')[-1])).astype('int16')
        grid_df['dept_id'] = grid_df['dept_id'].apply(lambda x: int(x.split('_')[-1])).astype('int16')
        grid_df['cat_id'] = grid_df['cat_id'].replace({'HOBBIES': 0, 'HOUSEHOLD': 1, 'FOODS': 2}).astype('int16')
        grid_df['snap_CA'] = grid_df['snap_CA'].astype('int8')
        grid_df['snap_TX'] = grid_df['snap_TX'].astype('int8')
        grid_df['snap_WI'] = grid_df['snap_WI'].astype('int8')
        for col in ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']:
            grid_df[col] = grid_df[col].replace(dict(zip(grid_df[col].unique(), np.arange(grid_df[col].unique().shape[0])))).astype('int32')
            
        grid_df.to_pickle(BASE_GRID)
    else:
        grid_df = pd.read_pickle(BASE_GRID)
    
    # Leave only relevant store
    grid_df = grid_df[grid_df['store_id']==store]
    return grid_df[BASE_FEATURES].reset_index(drop=True)

In [5]:
# grid_df = get_data_by_store(TEST_STORE_ID)

In [6]:
# grid_df[TS_COLUMNS].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4788267 entries, 0 to 4788266
Data columns (total 16 columns):
 #   Column        Dtype  
---  ------        -----  
 0   sell_price    float16
 1   sales         float64
 2   event_name_1  int32  
 3   event_type_1  int32  
 4   event_name_2  int32  
 5   event_type_2  int32  
 6   snap_CA       int8   
 7   snap_TX       int8   
 8   snap_WI       int8   
 9   tm_d          int8   
 10  tm_w          int8   
 11  tm_m          int8   
 12  tm_y          int8   
 13  tm_wm         int8   
 14  tm_dw         int8   
 15  tm_w_end      int8   
dtypes: float16(1), float64(1), int32(4), int8(10)
memory usage: 164.4 MB


In [6]:
# window_size = 56
def gen_dataset(series, window_size):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size))
    dataset = dataset.map(lambda window: (window[:-28], window[-28:]))
    return dataset

def get_train_valid(grid, ts_columns, static_columns, window_size = 56):
    
    X_train, y_train = [],[]
    X_train2 = []
    for id_, group in tqdm(grid.groupby('id')):
        if group.shape[0] == 0:
            continue
        train_dataset = gen_dataset(group[ts_columns].values, window_size)
        old_n_X_train = len(X_train)
        idx = group['sales'].shape[0]-window_size+1
        train_dataset_len = 0
        for x, y in train_dataset:
            X_train.append(x.numpy())# = np.vstack([X_train, x.numpy()])
            y_train.append(y.numpy())# = np.vstack([y_train, y.numpy()])
            train_dataset_len += 1
        
        sub_group = group.iloc[window_size//2-1:idx-1+window_size//2]
        if (len(X_train)-old_n_X_train) != (sub_group.shape[0]):
            print(id_, len(X_train)-old_n_X_train, sub_group.shape[0], group.shape[0])
        else:
            X_train2.append(sub_group[static_columns])
    
    X_train = np.array(X_train)
    y_train = np.array(y_train)[:,:,1:2]
    X_train2 = np.vstack(X_train2)
    
    return X_train, y_train, X_train2

In [7]:
# y_part1 = np.load(f'{BASE_PATH}/y_part1_{STORE_CA3}.npy')
# y_part1 = y_part1[:,:,1:2]
# y_part2 = y_part2[:,:,1:2]
# y_part3 = y_part3[:,:,1:2]
# y_part4 = y_part4[:,:,1:2]

In [5]:
if os.path.exists(f'{BASE_PATH}/X_part1_{TEST_STORE_ID}.npy'):
    X_part1 = np.load(f'{BASE_PATH}/X_part1_{TEST_STORE_ID}.npy')
    y_part1 = np.load(f'{BASE_PATH}/y_part1_{TEST_STORE_ID}.npy')
    X_part2 = np.load(f'{BASE_PATH}/X_part2_{TEST_STORE_ID}.npy')
    y_part2 = np.load(f'{BASE_PATH}/y_part2_{TEST_STORE_ID}.npy')
    X_part3 = np.load(f'{BASE_PATH}/X_part3_{TEST_STORE_ID}.npy')
    y_part3 = np.load(f'{BASE_PATH}/y_part3_{TEST_STORE_ID}.npy')
    X_part4 = np.load(f'{BASE_PATH}/X_part4_{TEST_STORE_ID}.npy')
    y_part4 = np.load(f'{BASE_PATH}/y_part4_{TEST_STORE_ID}.npy')
    
else:

    mask_part1 = (grid_df['d']>START_TRAIN)&(grid_df['d']<=END_TRAIN-2*part_len)
    mask_part2 = (grid_df['d']>START_TRAIN+part_len)&(grid_df['d']<=END_TRAIN-part_len)
    mask_part3 = (grid_df['d']>START_TRAIN+2*part_len)&(grid_df['d']<=END_TRAIN)
    mask_part4 = (grid_df['d']>END_TRAIN-28-28)&(grid_df['d']<=END_TRAIN)

    X_part1, y_part1, _ = get_train_valid(grid_df[mask_part1], TS_COLUMNS, STATIC_COLUMNS)
    X_part2, y_part2, _ = get_train_valid(grid_df[mask_part2], TS_COLUMNS, STATIC_COLUMNS)
    X_part3, y_part3, _ = get_train_valid(grid_df[mask_part3], TS_COLUMNS, STATIC_COLUMNS)
    X_part4, y_part4, _ = get_train_valid(grid_df[mask_part4],TS_COLUMNS, STATIC_COLUMNS)

    np.save(f'{BASE_PATH}/X_part1_{TEST_STORE_ID}.npy', X_part1) # save
    np.save(f'{BASE_PATH}/y_part1_{TEST_STORE_ID}.npy', y_part1) # save
    np.save(f'{BASE_PATH}/X_part2_{TEST_STORE_ID}.npy', X_part2) # save
    np.save(f'{BASE_PATH}/y_part2_{TEST_STORE_ID}.npy', y_part2) # save
    np.save(f'{BASE_PATH}/X_part3_{TEST_STORE_ID}.npy', X_part3) # save
    np.save(f'{BASE_PATH}/y_part3_{TEST_STORE_ID}.npy', y_part3) # save
    np.save(f'{BASE_PATH}/X_part4_{TEST_STORE_ID}.npy', X_part4) # save
    np.save(f'{BASE_PATH}/y_part4_{TEST_STORE_ID}.npy', y_part4) # save

In [6]:
X_train = np.vstack([X_part1, X_part2])
y_train = np.vstack([y_part1, y_part2])
X_valid = X_part3
y_valid = y_part3
X_valid2 = X_part4
y_valid2 = y_part4
train_sample_index = np.random.choice(X_train.shape[0], 500_000)
valid_sample_index = np.random.choice(X_valid.shape[0], 200_000)
X_train = X_train[train_sample_index].astype('float32')
y_train = y_train[train_sample_index].astype('float32')
X_valid = X_valid[valid_sample_index].astype('float32')
y_valid = y_valid[valid_sample_index].astype('float32')
X_valid2 = X_valid2.astype('float32')
y_valid2 = y_valid2.astype('float32')

In [7]:
import pickle 
from keras.optimizers import Adam
import numpy as np
from keras.models import Model
from keras.layers import Input, SimpleRNN, Dense, GRU, LSTM, Reshape, Lambda
from keras import backend as K

class TSBase:
    def summary(self):
        return self.model.summary()
    
    def compile(self, optimizer=Adam(1e-4), loss='mse', metrics=['mse']):
        self.model.compile(optimizer=optimizer, loss='mse')
    
    def save(self,model_name):
        self.model.save(model_name+'.h5')
        pickle.dump(self.class_info,open(model_name+'.info','wb'))

    def data_preprocessing(self,x, y = None):
        # overload this method if it doesn't fit your need. 
        x_new = x
        y_new = y

        return x_new, y_new

    def fit(self, x=None, y=None, batch_size=None, epochs=1, verbose=1, validation_data=None):
        x_train, y_train = self.data_preprocessing(x, y)
        if validation_data != None:
            x_test, y_test = self.data_preprocessing(validation_data[0], validation_data[1])
            validation_data = (x_test, y_test)

        self.history=self.model.fit(
        x_train, y_train,
        batch_size=batch_size,
        epochs=epochs,
        verbose=verbose,
        validation_data=validation_data,
        shuffle=False)

        return self.history
    
    def predict(self, x_test):
        x_test, _ = self.data_preprocessing(x_test)
        y_predict = self.model.predict(x_test)
        return y_predict
    
class Seq2Seq_2(TSBase):
    def __init__(self, input_shape, output_shape, cell, cell_units, reload = False ):
        """
            input_shape: shape of input data, (n_memory_steps, n_in_features)
            output_shape: shape of output data, (n_forcast_steps, n_out_features)
            cell: cell in the RNN part, 'SimpleRNN' / 'LSTM' / 'GRU'
            cell_units: number of hidden cell unit in RNN part, integer, e.g. 100
            reload: True if feed externally, False if generate from scratch 
        """
        if reload:
            self.model = None
            self.class_info = None
        else:
            # just for future reload
            self.class_info = {'class': 'Seq2Seq', 'input_shape': input_shape, 'output_shape': output_shape,
                'cell': cell, 'cell_units': cell_units}

            if cell == 'LSTM':
                # declare encoder and decoder objects
                encoder = LSTM(units = cell_units, return_state = True)
                decoder = LSTM(units = cell_units, return_sequences=True, return_state = True)
                decoder_dense = Dense(output_shape[-1])

                # data flow
                encoder_input = Input(input_shape)
                encoder_output, state_h, state_c = encoder(encoder_input)
                encoder_state = [state_h, state_c]  
                
                # initial input and state for iteration
                iter_input = Reshape((1,output_shape[-1]))(decoder_dense(state_h))
                iter_state = encoder_state
                all_output = []

                for _ in range(output_shape[0]):
                    # Run the decoder on one timestep, output == state_h since only one time step
                    output, state_h, state_c = decoder(iter_input, initial_state=iter_state)
                    output = decoder_dense(output)

                    # Store the current prediction (we will concatenate all predictions later)
                    all_output.append(output)
                    
                    # Reinject the outputs and state for the next loop iteration
                    iter_input = output
                    iter_state = [state_h, state_c]

            elif cell == 'SimpleRNN':
                # declare encoder and decoder objects
                encoder = SimpleRNN(units = cell_units, return_state = True)
                decoder = SimpleRNN(units = cell_units, return_sequences=True, return_state = True)
                decoder_dense = Dense(output_shape[-1])

                # data flow
                encoder_input = Input(input_shape)
                encoder_output, state_h = encoder(encoder_input)
                encoder_state = state_h  
                
                # initial input and state for iteration
                iter_input = Reshape((1,output_shape[-1]))(decoder_dense(state_h))
                iter_state = encoder_state
                all_output = []

                for _ in range(output_shape[0]):
                    # Run the decoder on one timestep, output == state_h since only one time step
                    output, state_h = decoder(iter_input, initial_state=iter_state)
                    output = decoder_dense(output)

                    # Store the current prediction (we will concatenate all predictions later)
                    all_output.append(output)
                    
                    # Reinject the outputs and state for the next loop iteration
                    iter_input = output
                    iter_state = state_h

            elif cell == 'GRU':
                # declare encoder and decoder objects
                encoder = GRU(units = cell_units, return_state = True)
                decoder = GRU(units = cell_units, return_sequences=True, return_state = True)
                decoder_dense = Dense(output_shape[-1])

                # data flow
                encoder_input = Input(input_shape)
                encoder_output, state_h = encoder(encoder_input)
                encoder_state = state_h  
                
                # initial input and state for iteration
                iter_input = Reshape((1,output_shape[-1]))(decoder_dense(state_h))
                iter_state = encoder_state
                all_output = []

                for _ in range(output_shape[0]):
                    # Run the decoder on one timestep, output == state_h since only one time step
                    output, state_h = decoder(iter_input, initial_state=iter_state)
                    output = decoder_dense(output)

                    # Store the current prediction (we will concatenate all predictions later)
                    all_output.append(output)
                    
                    # Reinject the outputs and state for the next loop iteration
                    iter_input = output
                    iter_state = state_h

            # Concatenate all predictions
            decoder_output = Lambda(lambda x: K.concatenate(x, axis=1))(all_output)
            self.model = Model(encoder_input, decoder_output)

Using TensorFlow backend.


In [12]:
model = Seq2Seq_2(input_shape=(28,16), output_shape=(28,1), cell='LSTM', cell_units=64)
model.compile()

In [13]:
history = model.fit(X_train, y_train, batch_size=28, epochs=10, verbose=1, validation_data=(X_valid,y_valid))

Train on 500000 samples, validate on 200000 samples
Epoch 1/10
500000/500000 [==============================] - 1965s 4ms/step - loss: 12.1969 - val_loss: 6.6243
Epoch 2/10
500000/500000 [==============================] - 1913s 4ms/step - loss: 10.7111 - val_loss: 6.4525
Epoch 3/10
500000/500000 [==============================] - 1913s 4ms/step - loss: 10.2785 - val_loss: 6.3181
Epoch 4/10
500000/500000 [==============================] - 1919s 4ms/step - loss: 10.0152 - val_loss: 6.2595
Epoch 5/10
311024/500000 [=================>............] - ETA: 10:02 - loss: 9.7748

KeyboardInterrupt: 

In [30]:
model.fit(X_train,y_train, epochs=1, verbose=10)

Epoch 1 Batch 0
batch loss Tensor("truediv_16:0", shape=(), dtype=float32)


KeyboardInterrupt: 

In [11]:
model = build_graph(64, 28, 16, n_mlp_features=None, weights_path=None)
model.summary()




Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


AttributeError: in converted code:

    <ipython-input-10-3e083c6e9192>:52 call
        score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))
    C:\Users\wangz\.conda\envs\tf_gpu1.14\lib\site-packages\tensorflow\python\keras\engine\base_layer.py:663 __call__
        inputs, outputs, args, kwargs)
    C:\Users\wangz\.conda\envs\tf_gpu1.14\lib\site-packages\tensorflow\python\keras\engine\base_layer.py:1708 _set_connectivity_metadata_
        input_tensors=inputs, output_tensors=outputs, arguments=kwargs)
    C:\Users\wangz\.conda\envs\tf_gpu1.14\lib\site-packages\tensorflow\python\keras\engine\base_layer.py:1795 _add_inbound_node
        input_tensors)
    C:\Users\wangz\.conda\envs\tf_gpu1.14\lib\site-packages\tensorflow\python\util\nest.py:515 map_structure
        structure[0], [func(*x) for x in entries],
    C:\Users\wangz\.conda\envs\tf_gpu1.14\lib\site-packages\tensorflow\python\util\nest.py:515 <listcomp>
        structure[0], [func(*x) for x in entries],


In [21]:
X_train = np.vstack([X_part1, X_part2])
y_train = np.vstack([y_part1, y_part2])
X_valid = X_part3
y_valid = y_part3
X_valid2 = X_part4
y_valid2 = y_part4
train_sample_index = np.random.choice(X_train.shape[0], 500_000)
valid_sample_index = np.random.choice(X_valid.shape[0], 200_000)


In [34]:
his = model.fit([X_train[train_sample_index],X_train[train_sample_index]],y_train[train_sample_index], validation_data=([X_valid[valid_sample_index],X_valid[valid_sample_index]], y_valid[valid_sample_index]), batch_size=14, epochs=1)
score = model.evaluate([X_valid2,X_valid2], y_valid2)
print(list(zip(model.metrics_names, score)))

Train on 500000 samples, validate on 200000 samples
Epoch 1/1
3049/3049 [==============================] - 3s 1ms/step
[('loss', 4.647151498882135), ('root_mean_squared_error', 2.987672805786133)]


In [ ]:
seq2seq
Train on 500000 samples, validate on 200000 samples
Epoch 1/1
500000/500000 [==============================] - 1252s 3ms/step - loss: 10.5394 - root_mean_squared_error: 3.3513 - val_loss: 5.8176 - val_root_mean_squared_error: 3.1287
3049/3049 [==============================] - 1s 370us/step
[('loss', 4.872688117985962), ('root_mean_squared_error', 3.030351400375366)]

In [ ]:
bi seq2seq
Train on 500000 samples, validate on 200000 samples
Epoch 1/1
500000/500000 [==============================] - 3484s 7ms/step - loss: 10.2177 - root_mean_squared_error: 3.2851 - val_loss: 5.7234 - val_root_mean_squared_error: 3.0826
3049/3049 [==============================] - 3s 1ms/step
[('loss', 4.647151498882135), ('root_mean_squared_error', 2.987672805786133)]

In [142]:
def gru(units):
  # If you have a GPU, we recommend using CuDNNGRU(provides a 3x speedup than GRU)
  # the code automatically does that.
  if tf.test.is_gpu_available():
    return tf.keras.layers.CuDNNGRU(units, 
                                    return_sequences=True, 
                                    return_state=True, 
                                    recurrent_initializer='glorot_uniform')
  else:
    return tf.keras.layers.GRU(units, 
                               return_sequences=True, 
                               return_state=True, 
                               recurrent_activation='sigmoid', 
                               recurrent_initializer='glorot_uniform')

In [143]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.enc_units)
        
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)        
        return output, state
    
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [144]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.dec_units)
        self.fc = tf.keras.layers.Dense(vocab_size)
        
        # used for attention
        self.W1 = tf.keras.layers.Dense(self.dec_units)
        self.W2 = tf.keras.layers.Dense(self.dec_units)
        self.V = tf.keras.layers.Dense(1)
        
    def call(self, x, hidden, enc_output):
        # enc_output shape == (batch_size, max_length, hidden_size)
        
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        
        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying tanh(FC(EO) + FC(H)) to self.V
        score = self.V(tf.nn.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis)))
        
        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
        
        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * enc_output
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)
        
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        
        # passing the concatenated vector to the GRU
        output, state = self.gru(x)
        
        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))
        
        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)
        
        return x, state, attention_weights
        
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.dec_units))

In [ ]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)
optimizer = tf.train.AdamOptimizer()


def loss_function(real, pred):
    mask = 1 - np.equal(real, 0)
    loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
    return tf.reduce_mean(loss_)

In [114]:
# window_size = 56
def gen_dataset(series, window_size):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size))
    dataset = dataset.map(lambda window: (window[:-28], window[-28:]))
    return dataset

def get_train_valid(grid, window_size = 56):
    
    X_train, y_train = [],[]
    X_train2 = []
    for id_, group in tqdm(grid.groupby('id')):
        if group.shape[0] == 0:
            continue
        train_dataset = gen_dataset(group[['sales']].values, window_size)
        old_n_X_train = len(X_train)
        idx = group['sales'].shape[0]-window_size+1
        train_dataset_len = 0
        for x, y in train_dataset:
            X_train.append(x.numpy())# = np.vstack([X_train, x.numpy()])
            y_train.append(y.numpy())# = np.vstack([y_train, y.numpy()])
            train_dataset_len += 1
        
        if (len(X_train)-old_n_X_train) != (group.iloc[window_size//2:idx+window_size//2].shape[0]):
            print(id_, len(X_train)-old_n_X_train, group.iloc[window_size//2:idx+window_size//2].shape[0], group.shape[0])
        else:
            X_train2.append(group.iloc[window_size//2-1:idx-1+window_size//2][MLP_COLUMNS])
#             break
    
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_train2 = np.vstack(X_train2)
    
    return X_train, y_train, X_train2


def build_graph(latent_dim, n_timeseries, n_mlp_features, weights_path=None):
    
    # Define an input sequence and process it.
    encoder_inputs = Input(shape=(28, 1))
    encoder = LSTM(latent_dim, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    # We discard `encoder_outputs` and only keep the states.
    encoder_states = [state_h, state_c]

    # Set up the decoder, using `encoder_states` as initial state.
    decoder_inputs = Input(shape=(28, 1))
    # We set up our decoder to return full output sequences,
    # and to return internal states as well. We don't use the
    # return states in the training model, but we will use them in inference.
    decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = Dense(1, activation='relu')
    decoder_outputs = decoder_dense(decoder_outputs)

    # Define the model that will turn
    # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

    # Run training
    model.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()])
    # model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
    #           batch_size=batch_size,
    #           epochs=epochs,
    #           validation_split=0.2)
    
#     if type(None) != type(weights_path):
#         model.load_weights(weights_path)

    return model


def train_model(X_train, X2_train, y_train, X_valid, X2_valid, y_valid, store_id, base_path, n_timeseries, n_mlp_features, batch_size=10, epochs=1):
    model = build_graph(n_timeseries, n_mlp_features, None)
    his = model.fit([X_train,X2_train], y_train, validation_data=([X_valid,X2_valid], y_valid), batch_size=batch_size, epochs=epochs, verbose=1)
    model.save_weights(f'{base_path}/bilstm_{store_id}.h5')
    return model

# def predict_series(model, series, mlp_input, n_timeseries):
    
#     lst = []
#     input1 = series
#     input2 = mlp_input
#     for i in range(29):
#         pred = model.predict([input1.reshape(1,n_timeseries), input2])
#         input1 = np.hstack((input1,pred[0]))[-n_timeseries:]
#         lst.append(pred[0][0])
#     return np.array(lst)

def predict_samples(grid_df, model):
    pred_df = pd.DataFrame(columns=['id']+[f'F{i}' for i in range(1,29)])
    idx = 0
    for id_, group in tqdm(grid_df.groupby('id')):
        if group.shape[0] == 0:
            continue
        input1 = group[group['d']<=1913]['sales'].values[-28:].reshape(1,28)
        input2 = group[group['d']<=1913].iloc[-1][MLP_COLUMNS].values.reshape(1,39)
        predictions = model.predict([input1.astype(float), input2.astype(float)])
        pred_df.loc[idx] = dict(zip(pred_df.columns.tolist(),[id_,*predictions[0]]))
        idx += 1
    return pred_df

def rmse(y, y_pred):
    return np.sqrt(np.mean(np.square(y - y_pred)))

In [32]:
from keras.models import Model, load_model
from keras.layers import Input
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import tensorflow as tf


In [20]:
store_id= 'CA_3'
X_part1 = np.load(f'{SAV_BASE_PATH}/X_part1_{store_id}.npy') # load
X2_part1 = np.load(f'{SAV_BASE_PATH}/X2_part1_{store_id}.npy') # load
y_part1 = np.load(f'{SAV_BASE_PATH}/X_part1_{store_id}.npy') # load
X_part2 = np.load(f'{SAV_BASE_PATH}/X_part2_{store_id}.npy') # load
X2_part2 = np.load(f'{SAV_BASE_PATH}/X2_part2_{store_id}.npy') # load
y_part2 = np.load(f'{SAV_BASE_PATH}/X_part2_{store_id}.npy') # load
X_part3 = np.load(f'{SAV_BASE_PATH}/X_part3_{store_id}.npy') # load
X2_part3 = np.load(f'{SAV_BASE_PATH}/X2_part3_{store_id}.npy') # load
y_part3 = np.load(f'{SAV_BASE_PATH}/X_part3_{store_id}.npy') # load    

In [ ]:
part4 = (grid_df['d']>END_TRAIN-28-28)&(grid_df['d']<=END_TRAIN)

X_part4, y_part4, X2_part4 = get_train_valid(grid_df[part4])

In [40]:
X_train = np.expand_dims(np.vstack([X_part1, X_part2]),-1)
X2_train = np.vstack([X2_part1, X2_part2])
y_train = np.expand_dims(np.vstack([y_part1, y_part2]),-1)
X_valid = np.expand_dims(X_part3,-1)
X2_valid = X2_part3
y_valid = np.expand_dims(y_part3,-1)
train_sample_index = np.random.choice(X_train.shape[0], 500_000)
valid_sample_index = np.random.choice(X_valid.shape[0], 200_000)

In [150]:
model = build_graph(64, 28, n_mlp_features=None, weights_path=None)
model.summary()

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           (None, 28, 1)        0                                            
__________________________________________________________________________________________________
input_17 (InputLayer)           (None, 28, 1)        0                                            
__________________________________________________________________________________________________
lstm_15 (LSTM)                  [(None, 64), (None,  16896       input_16[0][0]                   
__________________________________________________________________________________________________
lstm_16 (LSTM)                  [(None, 28, 64), (No 16896       input_17[0][0]                   
                                                                 lstm_15[0][1]              

In [42]:
model.fit([X_train[train_sample_index],X_train[train_sample_index]],y_train[train_sample_index], validation_data=([X_valid[valid_sample_index],X_valid[valid_sample_index]], y_valid[valid_sample_index]), batch_size=14)

Train on 500000 samples, validate on 200000 samples
Epoch 1/1
500000/500000 [==============================] - 1195s 2ms/step - loss: 5.5962 - root_mean_squared_error: 3.0428 - val_loss: 0.2000 - val_root_mean_squared_error: 2.1743


In [ ]:
BASE_GRID_DF = pd.read_pickle(f'{SAV_BASE_PATH}/BASE_FEATURES.pkl')
grid_df = BASE_GRID_DF[(BASE_GRID_DF['store_id']==store_id)]
del BASE_GRID_DF

In [76]:
def predict_samples(grid_df, model, endpoint=1913):
    pred_df = pd.DataFrame(columns=['id']+[f'F{i}' for i in range(1,29)])
    idx = 0
    for id_, group in tqdm(grid_df.groupby('id')):
        if group.shape[0] == 0:
            continue
        input1 = group[group['d']<=endpoint]['sales'].values[-28:].reshape(1,28,1)
#         input2 = group[group['d']<=1913].iloc[-1][MLP_COLUMNS].values.reshape(1,39)
        predictions = model.predict([input1.astype(float), input1.astype(float)])
        pred_df.loc[idx] = dict(zip(pred_df.columns.tolist(),[id_,*predictions.reshape(-1)]))
        idx += 1
    return pred_df

# pred_df = predict_samples(grid_df, model)

In [142]:
def get_train_valid(grid, window_size = 56):
    
    X_train, y_train = [],[]
    X_train2 = []
    for id_, group in tqdm(grid.groupby('id')):
        if group.shape[0] == 0:
            continue
        train_dataset = gen_dataset(group[['sales']].values, window_size)
        old_n_X_train = len(X_train)
        idx = group['sales'].shape[0]-window_size+1
        train_dataset_len = 0
        for x, y in train_dataset:
            X_train.append(x.numpy())# = np.vstack([X_train, x.numpy()])
            y_train.append(y.numpy())# = np.vstack([y_train, y.numpy()])
            train_dataset_len += 1
        
        if (len(X_train)-old_n_X_train) != (group.iloc[window_size//2:idx+window_size//2].shape[0]):
            print(id_, len(X_train)-old_n_X_train, group.iloc[window_size//2:idx+window_size//2].shape[0], group.shape[0])
        else:
            X_train2.append(group.iloc[window_size//2-1:idx-1+window_size//2][MLP_COLUMNS])
#             break
    
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_train2 = np.vstack(X_train2)
    
    return X_train, y_train, X_train2

In [143]:
part4 = (grid_df['d']>END_TRAIN-28-28)&(grid_df['d']<=END_TRAIN)

X_part4, y_part4, X2_part4 = get_train_valid(grid_df[part4])

100%|██████████| 30490/30490 [01:05<00:00, 462.49it/s]


In [144]:
X_part4.shape, y_part4.shape, X2_part4.shape

((3049, 28, 1), (3049, 28, 1), (3049, 39))

In [145]:
predictions = model.predict([X_part4.astype(float), X_part4.astype(float)])

In [149]:
rmse(y_part4.reshape(-1), predictions.reshape(-1))

3.756327027633038

In [ ]:
['event_name_1','event_type_1','event_name_2','event_type_2','snap_CA','snap_TX','snap_WI','tm_d','tm_w','tm_m','tm_y','tm_wm','tm_dw','tm_w_end,enc_store_id_tm_dw_item_id_mean','enc_store_id_tm_dw_item_id_std','enc_store_id_tm_dw_mean','enc_store_id_tm_dw_std']

In [129]:
pd.set_option('max_columns', 200)

In [130]:
grid_df[part4][grid_df[part4]['id']=='FOODS_1_001_CA_3_validation']

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,release,sell_price,price_max,price_min,price_std,price_mean,price_norm,price_nunique,item_nunique,price_momentum,price_momentum_m,price_momentum_y,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,tm_d,tm_w,tm_m,tm_y,tm_wm,tm_dw,tm_w_end,enc_store_id_cat_id_mean,enc_store_id_cat_id_std,enc_store_id_dept_id_mean,enc_store_id_dept_id_std,enc_store_id_item_id_mean,enc_store_id_item_id_std,enc_store_id_tm_dw_item_id_mean,enc_store_id_tm_dw_item_id_std,enc_store_id_tm_dw_mean,enc_store_id_tm_dw_std
44328227,FOODS_1_001_CA_3_validation,1,1,2,CA_3,CA,1858,1.0,0,2.240234,2.240234,1.75,0.115845,2.158203,1.0,3.0,68,1.0,1.037109,1.0,0,0,0,0,0,0,0,29,9,2,5,5,0,0,3.400391,9.085938,2.103516,3.941406,1.208008,2.34375,1.222656,2.265625,2.416016,6.773438
44358717,FOODS_1_001_CA_3_validation,1,1,2,CA_3,CA,1859,2.0,0,2.240234,2.240234,1.75,0.115845,2.158203,1.0,3.0,68,1.0,1.037109,1.0,0,0,0,0,1,1,0,1,9,3,5,1,1,0,3.400391,9.085938,2.103516,3.941406,1.208008,2.34375,1.022461,1.820312,2.208984,6.210938
44389207,FOODS_1_001_CA_3_validation,1,1,2,CA_3,CA,1860,0.0,0,2.240234,2.240234,1.75,0.115845,2.158203,1.0,3.0,68,1.0,1.037109,1.0,0,0,0,0,1,0,1,2,9,3,5,1,2,0,3.400391,9.085938,2.103516,3.941406,1.208008,2.34375,0.843750,1.856445,2.134766,6.031250
44419697,FOODS_1_001_CA_3_validation,1,1,2,CA_3,CA,1861,0.0,0,2.240234,2.240234,1.75,0.115845,2.158203,1.0,3.0,68,1.0,1.037109,1.0,0,0,0,0,1,1,1,3,9,3,5,1,3,0,3.400391,9.085938,2.103516,3.941406,1.208008,2.34375,0.829102,1.476562,2.111328,6.085938
44450187,FOODS_1_001_CA_3_validation,1,1,2,CA_3,CA,1862,1.0,0,2.240234,2.240234,1.75,0.115845,2.158203,1.0,3.0,68,1.0,1.037109,1.0,0,0,0,0,1,0,0,4,9,3,5,1,4,0,3.400391,9.085938,2.103516,3.941406,1.208008,2.34375,1.279297,2.527344,2.289062,6.824219
44480677,FOODS_1_001_CA_3_validation,1,1,2,CA_3,CA,1863,0.0,0,2.240234,2.240234,1.75,0.115845,2.158203,1.0,3.0,68,1.0,1.037109,1.0,0,0,0,0,1,1,1,5,9,3,5,1,5,1,3.400391,9.085938,2.103516,3.941406,1.208008,2.34375,1.421875,2.447266,2.757812,7.976562
44511167,FOODS_1_001_CA_3_validation,1,1,2,CA_3,CA,1864,1.0,0,2.240234,2.240234,1.75,0.115845,2.158203,1.0,3.0,68,1.0,1.037109,1.0,0,0,0,0,1,1,1,6,9,3,5,1,6,1,3.400391,9.085938,2.103516,3.941406,1.208008,2.34375,1.836914,3.369141,2.882812,7.929688
44541657,FOODS_1_001_CA_3_validation,1,1,2,CA_3,CA,1865,0.0,0,2.240234,2.240234,1.75,0.115845,2.158203,1.0,3.0,68,1.0,1.037109,1.0,0,0,0,0,1,1,0,7,10,3,5,1,0,0,3.400391,9.085938,2.103516,3.941406,1.208008,2.34375,1.222656,2.265625,2.416016,6.773438
44572147,FOODS_1_001_CA_3_validation,1,1,2,CA_3,CA,1866,0.0,0,2.240234,2.240234,1.75,0.115845,2.158203,1.0,3.0,68,1.0,1.037109,1.0,0,0,0,0,1,0,1,8,10,3,5,2,1,0,3.400391,9.085938,2.103516,3.941406,1.208008,2.34375,1.022461,1.820312,2.208984,6.210938
44602637,FOODS_1_001_CA_3_validation,1,1,2,CA_3,CA,1867,1.0,0,2.240234,2.240234,1.75,0.115845,2.158203,1.0,3.0,68,1.0,1.037109,1.0,0,0,0,0,1,1,1,9,10,3,5,2,2,0,3.400391,9.085938,2.103516,3.941406,1.208008,2.34375,0.843750,1.856445,2.134766,6.031250


In [127]:
grid_df[part4][grid_df[part4]['id']=='FOODS_1_001_CA_3_validation'].iloc[28:29]

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,release,sell_price,...,enc_store_id_cat_id_mean,enc_store_id_cat_id_std,enc_store_id_dept_id_mean,enc_store_id_dept_id_std,enc_store_id_item_id_mean,enc_store_id_item_id_std,enc_store_id_tm_dw_item_id_mean,enc_store_id_tm_dw_item_id_std,enc_store_id_tm_dw_mean,enc_store_id_tm_dw_std
45181947,FOODS_1_001_CA_3_validation,1,1,2,CA_3,CA,1886,0.0,0,2.240234,...,3.400391,9.085938,2.103516,3.941406,1.208008,2.34375,1.222656,2.265625,2.416016,6.773438


In [110]:
X_part4

array([], dtype=float64)

In [103]:
X_valid.shape[0]

(1064443, 28, 1)

In [95]:
X_valid[-1:].shape

(1, 28, 1)

In [96]:
predictions = model.predict([X_valid[-1:].astype(float), X_valid[-1:].astype(float)])

In [100]:
rmse(y_true, predictions.reshape(-1))

ValueError: operands could not be broadcast together with shapes (85372,) (28,) 

In [77]:
pred_df = predict_samples(grid_df, model,1913-28)

100%|██████████| 30490/30490 [00:41<00:00, 729.43it/s]


In [78]:
pred_df

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_3_validation,0.945388,2.003579,0.000000,0.000000,0.988437,0.000000,0.992105,0.000000,0.000000,...,0.991595,2.004172,3.996796,1.987121,0.000000,0.994274,1.000470,1.003092,12.005028,0.000000
1,FOODS_1_002_CA_3_validation,0.947493,0.000000,0.000000,0.987979,0.000000,0.990681,0.997816,0.000000,0.000000,...,0.993240,0.000000,0.992987,0.000000,0.000000,0.000000,0.000000,0.991029,2.999773,0.988705
2,FOODS_1_003_CA_3_validation,1.980366,1.996166,1.997639,0.000000,0.000000,0.991572,3.000392,0.000000,0.988590,...,0.000000,0.992334,0.998410,0.000000,0.000000,0.000000,0.000000,0.991140,0.997473,1.001314
3,FOODS_1_004_CA_3_validation,2.979722,0.000000,1.994847,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,FOODS_1_005_CA_3_validation,0.968645,0.000000,1.999153,0.989044,2.001684,0.000000,0.992203,2.005431,1.997760,...,1.990899,0.991345,3.003465,0.000000,0.990334,5.002184,1.988181,11.984709,5.998434,5.018535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,HOUSEHOLD_2_512_CA_3_validation,0.000000,0.000000,0.983884,0.993348,0.000000,2.002082,0.000000,2.000955,0.991547,...,3.004490,0.000000,0.990324,0.000000,0.994336,0.999221,0.000000,0.994228,0.000000,0.000000
3045,HOUSEHOLD_2_513_CA_3_validation,1.981005,2.992184,0.989580,4.000650,2.983788,5.984475,0.997073,0.000000,0.000000,...,0.991001,0.000000,0.000000,2.003559,0.990593,1.001264,1.003189,0.000000,0.000000,2.004761
3046,HOUSEHOLD_2_514_CA_3_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3047,HOUSEHOLD_2_515_CA_3_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.990822,0.000000,...,0.992053,2.004418,0.995675,1.002987,1.003892,0.000000,0.000000,0.000000,0.000000,0.000000


In [79]:
grid_df2 = pd.melt(pred_df,
                  id_vars=['id'],
                  var_name='d',
                  value_name='sales')

In [81]:
grid_df2['d'] = grid_df2['d'].apply(lambda x: int(x[1:]))

In [87]:
pred = grid_df2.sort_values(by=['id','d'])['sales'].values

In [88]:
y_true = grid_df[(grid_df['d']>=1913-28)&(grid_df['d']<1913)][['id','d','sales']].sort_values(by=['id','d'])['sales'].values

In [90]:
rmse(y_true,pred)

3.974663110840904

In [71]:
lgb_baseline = pd.read_csv('../cache/submission/lgbm_baseline_submission.csv')

In [73]:
# submission = pd.read_csv(ORIGINAL+'sample_submission.csv')[['id']]
lgb_baseline = pd.concat([lgb_baseline[~lgb_baseline['id'].isin(pred_df.id.unique().tolist())], pred_df], axis=0)
# lgb_baseline = lgb_baseline.merge(pred_df, on=['id'], how='left').fillna(0)
lgb_baseline.to_csv('../cache/submission/replace_ca3_lgbm_baseline_submission.csv', index=False)

In [74]:
lgb_baseline

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.808594,0.732422,0.711914,0.752930,0.948242,1.083984,1.169922,0.881836,0.892578,...,0.874023,1.097656,1.051758,0.833984,0.799316,0.762695,0.802246,0.916016,1.062500,1.027344
1,HOBBIES_1_002_CA_1_validation,0.201782,0.193237,0.171875,0.196289,0.219727,0.270020,0.316406,0.243774,0.223022,...,0.215088,0.270996,0.280273,0.204346,0.193115,0.185791,0.198975,0.205078,0.293213,0.301270
2,HOBBIES_1_003_CA_1_validation,0.426758,0.391602,0.408691,0.403809,0.549316,0.682617,0.619629,0.472656,0.456543,...,0.557617,0.660156,0.687012,0.510254,0.444824,0.475830,0.473633,0.601562,0.718262,0.665527
3,HOBBIES_1_004_CA_1_validation,1.681641,1.412109,1.414062,1.511719,1.986328,2.773438,3.113281,1.733398,1.427734,...,1.881836,2.589844,3.101562,1.763672,1.533203,1.421875,1.422852,1.927734,2.863281,3.267578
4,HOBBIES_1_005_CA_1_validation,0.958008,0.875000,0.896484,0.942383,1.068359,1.379883,1.486328,1.011719,1.024414,...,1.059570,1.470703,1.487305,1.005859,0.887695,0.928711,0.895996,1.125000,1.462891,1.457031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,HOUSEHOLD_2_512_CA_3_validation,0.000000,3.988287,0.000000,0.000000,0.000000,0.989662,2.003428,1.996493,0.998099,...,0.996859,2.006725,0.000000,0.000000,5.974878,0.000000,4.964254,0.982502,2.995314,1.989989
3045,HOUSEHOLD_2_513_CA_3_validation,0.000000,2.991708,0.978572,0.994755,0.998916,0.000000,0.993803,0.000000,2.004572,...,0.996056,0.000000,0.995169,0.000000,0.993438,2.005111,1.997271,3.997645,1.988478,4.997004
3046,HOUSEHOLD_2_514_CA_3_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.991025,...,0.990647,0.000000,0.000000,0.000000,0.991108,0.000000,0.000000,0.000000,0.000000,0.000000
3047,HOUSEHOLD_2_515_CA_3_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.990573,0.997179,1.001137,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [65]:
lgb_baseline

,id,F1_x,F2_x,F3_x,F4_x,F5_x,F6_x,F7_x,F8_x,F9_x,...,F19_y,F20_y,F21_y,F22_y,F23_y,F24_y,F25_y,F26_y,F27_y,F28_y
0,HOBBIES_1_001_CA_1_validation,0.808594,0.732422,0.711914,0.752930,0.948242,1.083984,1.169922,0.881836,0.892578,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,HOBBIES_1_002_CA_1_validation,0.201782,0.193237,0.171875,0.196289,0.219727,0.270020,0.316406,0.243774,0.223022,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_validation,0.426758,0.391602,0.408691,0.403809,0.549316,0.682617,0.619629,0.472656,0.456543,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,HOBBIES_1_004_CA_1_validation,1.681641,1.412109,1.414062,1.511719,1.986328,2.773438,3.113281,1.733398,1.427734,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,HOBBIES_1_005_CA_1_validation,0.958008,0.875000,0.896484,0.942383,1.068359,1.379883,1.486328,1.011719,1.024414,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60976,FOODS_3_824_WI_3_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60977,FOODS_3_825_WI_3_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60978,FOODS_3_826_WI_3_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
ca3_ids = []
for id_ in lgb_baseline.id.unique():
    if 'CA_3' in id_:
        ca3_ids.append(id_)

In [62]:
df_temp = lgb_baseline[lgb_baseline['id'].isin(ca3_ids)]

In [63]:
for col in [f'F{i}' for i in range(1,29)]:
    print(df_temp[col].sum())

5918.938812255859
5453.120452880859
5234.604309082031
5206.524627685547
5691.846588134766
7010.581939697266
7984.301086425781
6776.377227783203
6540.396392822266
6022.037933349609
5602.921966552734
6255.215881347656
7514.764434814453
7107.095291137695
6899.395782470703
6368.650009155273
5776.670211791992
5643.580017089844
5962.638031005859
7238.612091064453
7819.089630126953
6425.430725097656
5904.887588500977
5658.6055908203125
5572.8787841796875
5960.247650146484
7349.634429931641
7934.983856201172


In [54]:
for col in [f'F{i}' for i in range(1,29)]:
    print(pred_df[col].sum())

5462.296257972717
5474.489412546158
5189.987372756004
5463.501700758934
6553.12259221077
7198.065579891205
8387.158918738365
6610.26856815815
6210.309138774872
5675.529461622238
5629.015412688255
6183.9447066783905
7768.314437866211
8590.295038461685
6567.783604025841
5630.338264346123
5366.513480186462
5491.342837333679
5335.472414016724
6922.629359126091
8211.952028155327
6056.76638007164
5673.512939810753
5226.354175209999
5011.96452319622
5619.815213918686
7417.341598749161
7714.297100782394


In [11]:
# store_id = 'CA_1'
n_fold = 3
part_len = (END_TRAIN-START_TRAIN)//n_fold

In [12]:
STORE_IDS = pd.read_pickle(f'{SAV_BASE_PATH}/BASE_FEATURES.pkl').store_id.unique().tolist()

In [13]:
pred_df = pd.DataFrame()
for store_id in STORE_IDS:
    BASE_GRID_DF = pd.read_pickle(f'{SAV_BASE_PATH}/BASE_FEATURES.pkl')
    grid_df = BASE_GRID_DF[(BASE_GRID_DF['store_id']==store_id)]
    del BASE_GRID_DF
    part1 = (grid_df['d']>START_TRAIN)&(grid_df['d']<=END_TRAIN-2*part_len)
    part2 = (grid_df['d']>START_TRAIN+part_len)&(grid_df['d']<=END_TRAIN-part_len)
    part3 = (grid_df['d']>START_TRAIN+2*part_len)&(grid_df['d']<=END_TRAIN)

    X_part1, y_part1, X2_part1 = get_train_valid(grid_df[part1])
    X_part2, y_part2, X2_part2 = get_train_valid(grid_df[part2])
    X_part3, y_part3, X2_part3 = get_train_valid(grid_df[part3])
    
    np.save(f'{SAV_BASE_PATH}/X_part1_{store_id}.npy', X_part1) # save
    np.save(f'{SAV_BASE_PATH}/X2_part1_{store_id}.npy', X2_part1) # save
    np.save(f'{SAV_BASE_PATH}/y_part1_{store_id}.npy', y_part1) # save
    np.save(f'{SAV_BASE_PATH}/X_part2_{store_id}.npy', X_part2) # save
    np.save(f'{SAV_BASE_PATH}/X2_part2_{store_id}.npy', X2_part2) # save
    np.save(f'{SAV_BASE_PATH}/y_part2_{store_id}.npy', y_part2) # save
    np.save(f'{SAV_BASE_PATH}/X_part3_{store_id}.npy', X_part3) # save
    np.save(f'{SAV_BASE_PATH}/X2_part3_{store_id}.npy', X2_part3) # save
    np.save(f'{SAV_BASE_PATH}/y_part3_{store_id}.npy', y_part3) # save
    
    # X_part1 = np.load(f'X_part1.npy') # load
    # y_part1 = np.load(f'y_part1.npy') # load
    # X_part2 = np.load(f'X_part2.npy') # load
    # y_part2 = np.load(f'y_part2.npy') # load
    # X_part3 = np.load(f'X_part3.npy') # load
    # y_part3 = np.load(f'y_part3.npy') # load
    
    X_train = np.vstack([X_part1, X_part2])
    X2_train = np.vstack([X2_part1, X2_part2])
    y_train = np.vstack([y_part1, y_part2])
    X_valid = X_part3
    X2_valid = X2_part3
    y_valid = y_part3
    train_sample_index = np.random.choice(X_train.shape[0], 500_000)
    valid_sample_index = np.random.choice(X_valid.shape[0], 200_000)
    
    model = train_model(X_train[train_sample_index][:,:], X2_train[train_sample_index], y_train[train_sample_index], 
                    X_valid[valid_sample_index][:,:], X2_valid[valid_sample_index], y_valid[valid_sample_index], 
                    store_id, SAV_BASE_PATH, 28, 39, epochs=20)
    
    pred_df_i = predict_samples(grid_df, model)
    pred_df = pd.concat([pred_df, pred_df_i], axis=0)
#     break

FOODS_2_007_CA_1_validation 0 27 55
FOODS_2_031_CA_1_validation 0 13 41
FOODS_2_032_CA_1_validation 0 13 41
FOODS_2_035_CA_1_validation 0 6 34
FOODS_3_044_CA_1_validation 0 6 34
FOODS_3_105_CA_1_validation 0 6 34
FOODS_3_205_CA_1_validation 0 6 34
FOODS_3_229_CA_1_validation 0 20 48
FOODS_3_270_CA_1_validation 0 13 41
FOODS_3_275_CA_1_validation 0 6 34
FOODS_3_359_CA_1_validation 0 27 55
FOODS_3_429_CA_1_validation 0 13 41
FOODS_3_482_CA_1_validation 0 27 55
FOODS_3_521_CA_1_validation 0 6 34
FOODS_3_557_CA_1_validation 0 13 41
FOODS_3_693_CA_1_validation 0 20 48
HOBBIES_1_219_CA_1_validation 0 27 55
HOBBIES_1_354_CA_1_validation 0 6 34
HOUSEHOLD_1_283_CA_1_validation 0 13 41
HOUSEHOLD_1_332_CA_1_validation 0 6 34
HOUSEHOLD_1_384_CA_1_validation 0 6 34
HOUSEHOLD_2_114_CA_1_validation 0 20 48
HOUSEHOLD_2_132_CA_1_validation 0 6 34
HOUSEHOLD_2_307_CA_1_validation 0 6 34
HOUSEHOLD_2_360_CA_1_validation 0 6 34
HOUSEHOLD_2_487_CA_1_validation 0 27 55
HOUSEHOLD_2_515_CA_1_validation 0 6 34
F

100%|█████████▉| 30361/30490 [02:02<00:00, 251.93it/s]

FOODS_3_418_CA_2_validation 0 6 34
FOODS_3_482_CA_2_validation 0 6 34
FOODS_3_557_CA_2_validation 0 6 34
FOODS_3_693_CA_2_validation 0 20 48
FOODS_3_761_CA_2_validation 0 13 41
HOBBIES_1_219_CA_2_validation 0 27 55
HOBBIES_1_334_CA_2_validation 0 6 34
HOBBIES_1_339_CA_2_validation 0 6 34
HOBBIES_1_354_CA_2_validation 0 6 34
HOUSEHOLD_1_018_CA_2_validation 0 20 48
HOUSEHOLD_1_022_CA_2_validation 0 13 41
HOUSEHOLD_1_267_CA_2_validation 0 20 48
HOUSEHOLD_1_283_CA_2_validation 0 6 34
HOUSEHOLD_1_332_CA_2_validation 0 20 48
HOUSEHOLD_1_384_CA_2_validation 0 13 41
HOUSEHOLD_2_132_CA_2_validation 0 6 34
HOUSEHOLD_2_175_CA_2_validation 0 6 34
HOUSEHOLD_2_177_CA_2_validation 0 13 41
HOUSEHOLD_2_307_CA_2_validation 0 13 41
FOODS_2_305_CA_2_validation 0 18 46
FOODS_3_003_CA_2_validation 0 25 53
FOODS_3_047_CA_2_validation 0 25 53
FOODS_3_119_CA_2_validation 0 25 53
FOODS_3_134_CA_2_validation 0 4 32
FOODS_3_147_CA_2_validation 0 18 46
FOODS_3_166_CA_2_validation 0 25 53
FOODS_3_169_CA_2_validatio

100%|██████████| 30490/30490 [02:10<00:00, 234.10it/s]


FOODS_2_096_CA_3_validation 0 20 48
FOODS_3_044_CA_3_validation 0 6 34
FOODS_3_084_CA_3_validation 0 27 55
FOODS_3_105_CA_3_validation 0 6 34
FOODS_3_218_CA_3_validation 0 6 34
FOODS_3_229_CA_3_validation 0 6 34
FOODS_3_270_CA_3_validation 0 20 48
FOODS_3_275_CA_3_validation 0 27 55
FOODS_3_359_CA_3_validation 0 27 55
FOODS_3_429_CA_3_validation 0 13 41
FOODS_3_482_CA_3_validation 0 27 55
FOODS_3_525_CA_3_validation 0 13 41
HOBBIES_1_219_CA_3_validation 0 13 41
HOBBIES_1_354_CA_3_validation 0 6 34
HOBBIES_2_041_CA_3_validation 0 27 55
HOUSEHOLD_1_018_CA_3_validation 0 20 48
HOUSEHOLD_1_332_CA_3_validation 0 20 48
HOUSEHOLD_1_384_CA_3_validation 0 6 34
HOUSEHOLD_1_435_CA_3_validation 0 13 41
HOUSEHOLD_1_450_CA_3_validation 0 6 34
HOUSEHOLD_1_495_CA_3_validation 0 6 34
HOUSEHOLD_2_005_CA_3_validation 0 6 34
HOUSEHOLD_2_060_CA_3_validation 0 13 41
HOUSEHOLD_2_114_CA_3_validation 0 20 48
HOUSEHOLD_2_132_CA_3_validation 0 6 34
HOUSEHOLD_2_175_CA_3_validation 0 20 48
HOUSEHOLD_2_177_CA_3_val

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



500000/500000 [==============================] - 325s 650us/sample - loss: 7.6717 - root_mean_squared_error: 2.7698 - val_loss: 6.3947 - val_root_mean_squared_error: 2.5288
Epoch 19/20
169070/500000 [=========>....................] - ETA: 2:55 - loss: 7.2058 - root_mean_squared_error: 2.6844

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



500000/500000 [==============================] - 327s 655us/sample - loss: 7.6675 - root_mean_squared_error: 2.7690 - val_loss: 6.3966 - val_root_mean_squared_error: 2.5291


100%|██████████| 30490/30490 [02:17<00:00, 221.05it/s]


In [14]:
for col in [f'F{i}' for i in range(1,29)]:
    print(col, pred_df[col].sum())

F1 40586.18937710277
F2 29395.143401602167
F3 38120.38545977829
F4 36982.47288805284
F5 33591.79892578081
F6 39587.439021511425
F7 21188.550194994838
F8 30484.770024415222
F9 35838.44382213248
F10 42286.74087745728
F11 27302.81191282044
F12 31864.939928364183
F13 36296.47553243057
F14 31806.939007062843
F15 36411.397060816176
F16 40450.16693109856
F17 17131.708616930875
F18 33480.58410119888
F19 23496.48269185616
F20 34811.43517611466
F21 38752.0814780636
F22 39937.21767022612
F23 27844.90706532309
F24 40845.706697984686
F25 25612.66726968481
F26 39288.13686922821
F27 39248.542891609424
F28 34933.17951326945


In [16]:
pred_df

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,0.0,0.928475,1.088036,0.856354,1.026586,0.547378,0.0,0.0,1.222985,...,1.036828,0.810852,0.573717,0.759222,0.0,1.016626,0.0,0.601889,0.0,0.689259
1,FOODS_1_002_CA_1_validation,0.0,0.455638,0.636054,0.360710,0.294070,0.092625,0.0,0.0,0.830625,...,0.361501,0.190376,0.128430,0.463296,0.0,0.762140,0.0,0.177825,0.0,0.411012
2,FOODS_1_003_CA_1_validation,0.0,0.553006,0.494741,0.323444,0.643956,0.142067,0.0,0.0,0.695731,...,0.676567,0.619101,0.481467,0.353034,0.0,0.724214,0.0,0.561560,0.0,0.398130
3,FOODS_1_004_CA_1_validation,0.0,3.015525,2.188738,1.849086,2.785250,2.958735,0.0,0.0,5.067468,...,4.352099,4.261330,5.083395,5.753962,0.0,4.373944,0.0,3.045892,0.0,5.907110
4,FOODS_1_005_CA_1_validation,0.0,1.465038,1.265729,1.287864,1.192603,1.236912,0.0,0.0,1.745727,...,1.239331,1.134817,1.429517,1.915966,0.0,1.489210,0.0,1.213023,0.0,1.360599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
60976,FOODS_3_824_WI_3_evaluation,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
60977,FOODS_3_825_WI_3_evaluation,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
60978,FOODS_3_826_WI_3_evaluation,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000


In [15]:
sample_submission = pd.read_csv(f'{ORIGINAL}/sample_submission.csv')
pred_df = pd.concat([pred_df, sample_submission[~sample_submission['id'].isin(pred_df.id.unique().tolist())]], axis=0)
pred_df.to_csv('../cache/submission/lstm_submission.csv', index=False)